# De Text Mining a Representaciones Semánticas: Un Puente Conceptual

**Tecnicatura Superior en Ciencias de Datos e Inteligencia Artificial**  
**Procesamiento de Lenguaje Natural**

---

## Introducción: Construyendo sobre Fundamentos Sólidos

En el laboratorio anterior trabajaron intensamente con los cuentos de Hernán Casciari, aplicando técnicas fundamentales de **text mining**: preprocesamiento, tokenización, eliminación de stop words, vectorización con **Bag of Words** (BoW), análisis de frecuencias, y visualización con nubes de palabras. Fue un trabajo riguroso que les permitió extraer patrones temporales reales de un corpus auténtico.

Ese laboratorio no fue solo un ejercicio técnico: fue una exploración profunda de cómo la vida de un escritor se refleja en su vocabulario a lo largo del tiempo. Descubrieron la evolución temática de Casciari, desde los primeros años centrados en experiencias personales hasta la consolidación de Orsai como proyecto editorial. Aprendieron a manejar matrices esparsas, a filtrar ruido textual, y a extraer información significativa de datos no estructurados.

**Hoy, sin embargo, vamos a hacer algo diferente.** En lugar de celebrar únicamente los éxitos de esa experiencia, vamos a examinar críticamente sus limitaciones. No para desvalorizar lo que hicieron, sino para entender por qué necesitamos herramientas más sofisticadas para capturar la riqueza semántica del lenguaje.

### Objetivo de Esta Clase

Esta clase funciona como un **puente conceptual** entre el text mining tradicional que ya dominan y las representaciones semánticas que explorarán en la práctica del jueves. Nuestros objetivos específicos son:

1. **Identificar y analizar las limitaciones semánticas** de BoW y TF-IDF que experimentaron con Casciari
2. **Introducir el concepto de similitud semántica** y demostrar por qué es crucial para NLP moderno
3. **Experimentar con vectores semánticos densos** usando spaCy como primer acercamiento
4. **Preparar el terreno conceptual** para word embeddings, Word2Vec y modelos similares
5. **Desarrollar intuición** sobre por qué las representaciones semánticas revolucionaron el campo

### Metodología: Aprendizaje por Contraste

Utilizaremos una metodología de **aprendizaje por contraste**: confrontaremos directamente las limitaciones de BoW/TF-IDF con las capacidades de representaciones semánticas. Usaremos ejemplos concretos, muchos extraídos del propio corpus de Casciari, para hacer visibles problemas que quizás sintieron intuitivamente pero no pudieron articular formalmente.

No se trata de reemplazar conocimiento, sino de **expandirlo y profundizarlo**.

---

## 1. Reconexión: El Corpus Casciari como Punto de Partida

### Recordando lo que Lograron

Antes de avanzar, reconozcamos la solidez del trabajo que realizaron con el corpus de Casciari. Su análisis reveló patrones temporales fascinantes:

- **Evolución temática**: El vocabulario de 2004-2007 (primeros años de paternidad) difería claramente del de 2010-2011 (años de Orsai)
- **Diversidad léxica**: Identificaron años con mayor riqueza vocabular y conectaron esos patrones con eventos biográficos
- **Palabras características**: Descubrieron términos que definían períodos específicos ("revista", "orsai", "papelitos")
- **Técnicas profesionales**: Manejaron matrices de 12×29,683 dimensiones, optimizaron stop words, y crearon visualizaciones informativas

Este trabajo les dio experiencia práctica con los fundamentos del NLP y una comprensión visceral de cómo funciona la vectorización de texto en escenarios reales.

### Un Dataset Familiar para Nuevos Experimentos

Hoy volveremos a usar elementos del corpus Casciari, pero con un propósito diferente: como laboratorio para explorar las limitaciones de nuestras herramientas actuales y la necesidad de enfoques más sofisticados.

In [ ]:
# Configuración inicial del entorno
import pandas as pd
import numpy as np
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# Configuración de visualización
plt.style.use('default')
sns.set_palette("husl")
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)

print("Entorno configurado. Iniciando exploración crítica de text mining.")

Entorno configurado. Iniciando exploración crítica de text mining.


In [ ]:
# Recreamos algunos ejemplos inspirados en el corpus Casciari para nuestros experimentos
# Estos fragmentos capturan el estilo y las temáticas que encontraron en su análisis

textos_casciari_inspirados = {
    "paternidad_temprana": """La nena se despertó a las tres de la mañana llorando.
    No sabía qué hacer, así que la alcé y caminé por la casa hasta que se calmó.
    Ser padre es lo más difícil que me ha tocado hacer en la vida.""",

    "paternidad_madura": """Mi hija se levantó temprano y me preparó el desayuno.
    Ya no es una bebé que llora por las noches, ahora es una persona
    que tiene sus propias opiniones sobre todo.""",

    "futbol_pasion": """El partido fue increíble. San Lorenzo jugó como nunca
    y la hinchada cantó durante los noventa minutos.
    El fútbol argentino tiene esa magia que no encuentro en ningún otro lugar.""",

    "futbol_nostalgia": """Viendo el encuentro por televisión recordé cuando iba
    a la cancha con mi viejo. La pasión azulgrana corría por nuestras venas
    y cada gol era una fiesta familiar.""",

    "orsai_nacimiento": """La idea de la revista surgió casi por casualidad.
    Quería hacer algo diferente, contar historias que no encontraba
    en otros medios. Los lectores respondieron de manera sorprendente.""",

    "orsai_consolidacion": """La publicación ya tiene su identidad propia.
    Los escritores que colaboran entienden el espíritu del proyecto
    y cada número es mejor que el anterior.""",

    "escritura_oficio": """Escribir es mi trabajo, pero también mi obsesión.
    Cada texto es una oportunidad de conectar con los lectores
    de una manera auténtica y directa.""",

    "escritura_arte": """Las palabras son mi herramienta para crear mundos.
    Cada relato es una invitación a los lectores para que
    construyan sus propias interpretaciones."""
}

# Convertir a DataFrame para facilitar manipulación
df_ejemplos = pd.DataFrame(list(textos_casciari_inspirados.items()),
                          columns=['categoria', 'texto'])

print(f"Preparados {len(df_ejemplos)} fragmentos temáticos para análisis")
print("\nCategorías disponibles:")
for cat in df_ejemplos['categoria'].unique():
    print(f"  - {cat}")

Preparados 8 fragmentos temáticos para análisis

Categorías disponibles:
  - paternidad_temprana
  - paternidad_madura
  - futbol_pasion
  - futbol_nostalgia
  - orsai_nacimiento
  - orsai_consolidacion
  - escritura_oficio
  - escritura_arte


---

## 2. Las Limitaciones que Experimentaron (Sin Saberlo)

### Problema 1: La Ceguera Semántica de BoW

Cuando analizaron el corpus de Casciari con Bag of Words, el modelo trataba cada palabra como una entidad completamente independiente. **"Padre" y "papá" eran tan diferentes entre sí como "padre" y "computadora".**

Esto significa que si Casciari usaba "padre" en 2004 y "papá" en 2008 para referirse al mismo concepto, su análisis de frecuencias no captaba esa continuidad temática. La riqueza semántica del lenguaje humano quedaba invisible.

Veamos este problema en acción:

In [ ]:
# Ejemplo concreto: sinónimos que BoW trata como palabras independientes
ejemplos_sinonimos = [
    "Mi papá me enseñó a jugar al fútbol cuando era chico",
    "Mi padre me mostró cómo patear la pelota de niño",
    "El viejo me explicó las reglas del juego siendo pequeño"
]

# Vectorización con BoW
vectorizer_bow = CountVectorizer()
matriz_bow = vectorizer_bow.fit_transform(ejemplos_sinonimos)
feature_names = vectorizer_bow.get_feature_names_out()

# Crear DataFrame para visualizar
df_bow = pd.DataFrame(matriz_bow.toarray(), columns=feature_names)
df_bow.index = ['Frase 1', 'Frase 2', 'Frase 3']

print("REPRESENTACIÓN BoW DE TRES FRASES SEMÁNTICAMENTE SIMILARES:")
print("=" * 65)
print(df_bow)
print("\n💭 OBSERVACIÓN CRÍTICA:")
print("Estas tres frases hablan del mismo recuerdo (el padre enseñando fútbol),")
print("pero BoW las ve como completamente diferentes porque no comparten palabras.")

# Calcular similitud coseno
similitudes = cosine_similarity(matriz_bow)
print("\nSIMILITUD COSENO ENTRE FRASES (según BoW):")
print(f"Frase 1 vs Frase 2: {similitudes[0,1]:.3f}")
print(f"Frase 1 vs Frase 3: {similitudes[0,2]:.3f}")
print(f"Frase 2 vs Frase 3: {similitudes[1,2]:.3f}")
print("\n⚠️  Similitud = 0 significa que BoW considera estas frases completamente diferentes")

REPRESENTACIÓN BoW DE TRES FRASES SEMÁNTICAMENTE SIMILARES:
         al  chico  cuando  cómo  de  del  el  enseñó  era  explicó  fútbol  \
Frase 1   1      1       1     0   0    0   0       1    1        0       1   
Frase 2   0      0       0     1   1    0   0       0    0        0       0   
Frase 3   0      0       0     0   0    1   1       0    0        1       0   

         juego  jugar  la  las  me  mi  mostró  niño  padre  papá  patear  \
Frase 1      0      1   0    0   1   1       0     0      0     1       0   
Frase 2      0      0   1    0   1   1       1     1      1     0       1   
Frase 3      1      0   0    1   1   0       0     0      0     0       0   

         pelota  pequeño  reglas  siendo  viejo  
Frase 1       0        0       0       0      0  
Frase 2       1        0       0       0      0  
Frase 3       0        1       1       1      1  

💭 OBSERVACIÓN CRÍTICA:
Estas tres frases hablan del mismo recuerdo (el padre enseñando fútbol),
pero BoW las ve c

### Problema 2: Pérdida del Contexto Semántico

En su análisis de Casciari probablemente notaron palabras como "tiempo" apareciendo con alta frecuencia. Pero BoW no distingue entre:

- "No tengo **tiempo** para escribir" (recurso escaso)
- "El **tiempo** pasa rápido cuando juego con mi hija" (experiencia subjetiva)
- "En **tiempo** de Navidad todo cambia" (período específico)

Para BoW, estos son simplemente tres ocurrencias de la palabra "tiempo", pero semánticamente representan conceptos diferentes.

In [ ]:
# Ejemplo de polisemia: una palabra, múltiples significados
frases_tiempo = [
    "No tengo tiempo para escribir hoy, estoy muy ocupado",
    "El tiempo vuela cuando estoy con mi familia",
    "En tiempo de pandemia cambió nuestra forma de vivir",
    "Hace tiempo que no veo una película tan buena",
    "El tiempo atmosférico está muy cambiante esta semana"
]

vectorizer_tiempo = CountVectorizer()
matriz_tiempo = vectorizer_tiempo.fit_transform(frases_tiempo)

# Buscar la columna correspondiente a "tiempo"
feature_names_tiempo = vectorizer_tiempo.get_feature_names_out()
indice_tiempo = np.where(feature_names_tiempo == 'tiempo')[0][0]

print("ANÁLISIS DE POLISEMIA: LA PALABRA 'TIEMPO'")
print("=" * 45)
print("\nFrecuencia de 'tiempo' en cada frase:")
for i, frase in enumerate(frases_tiempo):
    freq = matriz_tiempo[i, indice_tiempo]
    print(f"Frase {i+1}: {freq} ocurrencia(s)")
    print(f"  Contexto: {frase[:50]}...")

print("\n💭 REFLEXIÓN CRÍTICA:")
print("BoW cuenta 5 ocurrencias de 'tiempo', pero cada una tiene un significado diferente:")
print("  • Tiempo como recurso (frase 1)")
print("  • Tiempo como experiencia subjetiva (frase 2)")
print("  • Tiempo como período histórico (frase 3)")
print("  • Tiempo como duración (frase 4)")
print("  • Tiempo como condición climática (frase 5)")
print("\n⚠️  BoW/TF-IDF no puede distinguir entre estos usos semánticamente diferentes")

ANÁLISIS DE POLISEMIA: LA PALABRA 'TIEMPO'

Frecuencia de 'tiempo' en cada frase:
Frase 1: 1 ocurrencia(s)
  Contexto: No tengo tiempo para escribir hoy, estoy muy ocupa...
Frase 2: 1 ocurrencia(s)
  Contexto: El tiempo vuela cuando estoy con mi familia...
Frase 3: 1 ocurrencia(s)
  Contexto: En tiempo de pandemia cambió nuestra forma de vivi...
Frase 4: 1 ocurrencia(s)
  Contexto: Hace tiempo que no veo una película tan buena...
Frase 5: 1 ocurrencia(s)
  Contexto: El tiempo atmosférico está muy cambiante esta sema...

💭 REFLEXIÓN CRÍTICA:
BoW cuenta 5 ocurrencias de 'tiempo', pero cada una tiene un significado diferente:
  • Tiempo como recurso (frase 1)
  • Tiempo como experiencia subjetiva (frase 2)
  • Tiempo como período histórico (frase 3)
  • Tiempo como duración (frase 4)
  • Tiempo como condición climática (frase 5)

⚠️  BoW/TF-IDF no puede distinguir entre estos usos semánticamente diferentes


### Problema 3: Incapacidad para Capturar Relaciones Conceptuales

Su análisis de Casciari mostró la evolución de temas familiares a lo largo del tiempo. Sin embargo, BoW no puede entender que:

- **"bebé" → "nena" → "hija" → "adolescente"** representan la misma persona en diferentes etapas
- **"San Lorenzo" → "azulgrana" → "hinchada" → "cancha"** forman un campo semántico relacionado con fútbol
- **"escribir" → "texto" → "relato" → "historia"** están conceptualmente conectados

Esta limitación es crucial porque significa que patrones semánticos profundos permanecen invisibles en el análisis.

In [ ]:
# Demostración: campos semánticos que BoW no puede relacionar
campos_semanticos = {
    "Fútbol (explícito)": "El partido de fútbol fue emocionante, San Lorenzo ganó",
    "Fútbol (implícito 1)": "La hinchada cantó durante todo el encuentro azulgrana",
    "Fútbol (implícito 2)": "El Ciclón mostró gran juego y la tribuna se volvió loca",
    "Escritura (explícito)": "Escribir es mi pasión, cada texto es una aventura",
    "Escritura (implícito 1)": "La narración fluía y las palabras cobraban vida",
    "Escritura (implícito 2)": "El relato cautivó a los lectores desde el primer párrafo"
}

textos_campos = list(campos_semanticos.values())
labels_campos = list(campos_semanticos.keys())

# Vectorización
vectorizer_campos = TfidfVectorizer()
matriz_campos = vectorizer_campos.fit_transform(textos_campos)

# Calcular similitudes
similitudes_campos = cosine_similarity(matriz_campos)

print("SIMILITUD ENTRE TEXTOS DE CAMPOS SEMÁNTICOS RELACIONADOS")
print("=" * 60)

# Comparar textos dentro del mismo campo semántico
print("\n🏈 CAMPO SEMÁNTICO: FÚTBOL")
print(f"Explícito vs Implícito 1: {similitudes_campos[0,1]:.3f}")
print(f"Explícito vs Implícito 2: {similitudes_campos[0,2]:.3f}")
print(f"Implícito 1 vs Implícito 2: {similitudes_campos[1,2]:.3f}")

print("\n✍️  CAMPO SEMÁNTICO: ESCRITURA")
print(f"Explícito vs Implícito 1: {similitudes_campos[3,4]:.3f}")
print(f"Explícito vs Implícito 2: {similitudes_campos[3,5]:.3f}")
print(f"Implícito 1 vs Implícito 2: {similitudes_campos[4,5]:.3f}")

print("\n🔄 COMPARACIÓN ENTRE CAMPOS (debe ser baja):")
print(f"Fútbol explícito vs Escritura explícita: {similitudes_campos[0,3]:.3f}")

print("\n💭 ANÁLISIS CRÍTICO:")
print("- Las similitudes dentro de cada campo son bajas porque no comparten palabras exactas")
print("- TF-IDF no reconoce que 'hinchada' y 'tribuna' se refieren al mismo concepto")
print("- No entiende que 'relato', 'narración' y 'texto' están semánticamente relacionados")
print("- Patrones temáticos profundos quedan invisibles en el análisis")

### Problema 4: La Tragedia de los Sinónimos

Este es quizás el problema más frustrante que experimentaron sin darse cuenta. Casciari, como buen escritor, usa sinónimos para enriquecer su prosa. Pero para BoW/TF-IDF:

- **"hermoso" y "bello"** son palabras completamente diferentes
- **"auto" y "coche"** no tienen relación alguna  
- **"enojado" y "furioso"** podrían estar en galaxias diferentes

Esto significa que su análisis de frecuencias fragmentó artificialmente conceptos que deberían estar unidos.

In [ ]:
# Ejemplo dramático: la fragmentación de conceptos por sinónimos
textos_sinonimos = [
    "El paisaje era hermoso, me quedé contemplando la vista",
    "La vista era bella, no podía dejar de admirar el panorama",
    "Qué lindo lugar, la perspectiva desde aquí es preciosa",
    "Este sitio es precioso, la panorámica es realmente bonita"
]

# Análisis con TF-IDF
vectorizer_sin = TfidfVectorizer()
matriz_sin = vectorizer_sin.fit_transform(textos_sinonimos)
feature_names_sin = vectorizer_sin.get_feature_names_out()

# Crear DataFrame para análisis
df_sinonimos = pd.DataFrame(matriz_sin.toarray(), columns=feature_names_sin)
df_sinonimos.index = [f'Descripción {i+1}' for i in range(len(textos_sinonimos))]

print("ANÁLISIS DE SINÓNIMOS: CONCEPTOS FRAGMENTADOS")
print("=" * 50)

# Identificar palabras relacionadas con belleza
palabras_belleza = ['hermoso', 'bella', 'lindo', 'preciosa', 'precioso', 'bonita']
palabras_vista = ['vista', 'panorama', 'perspectiva', 'panorámica']

print("\n🎨 PALABRAS DE BELLEZA EN EL CORPUS:")
for palabra in palabras_belleza:
    if palabra in feature_names_sin:
        idx = np.where(feature_names_sin == palabra)[0][0]
        ocurrencias = df_sinonimos.iloc[:, idx].values
        frases_con_palabra = [i for i, val in enumerate(ocurrencias) if val > 0]
        print(f"'{palabra}': aparece en descripción(es) {[f+1 for f in frases_con_palabra]}")

print("\n🔭 PALABRAS DE PERSPECTIVA VISUAL:")
for palabra in palabras_vista:
    if palabra in feature_names_sin:
        idx = np.where(feature_names_sin == palabra)[0][0]
        ocurrencias = df_sinonimos.iloc[:, idx].values
        frases_con_palabra = [i for i, val in enumerate(ocurrencias) if val > 0]
        print(f"'{palabra}': aparece en descripción(es) {[f+1 for f in frases_con_palabra]}")

# Calcular similitudes
similitudes_sin = cosine_similarity(matriz_sin)

print("\n📊 SIMILITUDES ENTRE DESCRIPCIONES:")
for i in range(len(textos_sinonimos)):
    for j in range(i+1, len(textos_sinonimos)):
        sim = similitudes_sin[i,j]
        print(f"Descripción {i+1} vs {j+1}: {sim:.3f}")

print("\n💭 TRAGEDIA DE LOS SINÓNIMOS:")
print("- Las 4 descripciones hablan del mismo concepto: un lugar hermoso con buena vista")
print("- Pero TF-IDF las ve como textos diferentes porque usan sinónimos")
print("- 'Hermoso', 'bello', 'lindo', 'precioso' expresan la misma idea")
print("- 'Vista', 'panorama', 'perspectiva' se refieren al mismo elemento")
print("- La riqueza léxica del español se convierte en una limitación técnica")

---

## 3. Hacia la Semántica Computacional: Una Nueva Perspectiva

### El Salto Conceptual Necesario

Los problemas que acabamos de identificar no son fallas de implementación que pueden corregirse con mejores algoritmos de preprocesamiento o stop words más sofisticadas. **Son limitaciones fundamentales de la representación bag-of-words.**

Para superarlas, necesitamos un cambio de paradigma: pasar de representaciones basadas en **co-ocurrencia de palabras exactas** a representaciones que capturen **significado semántico**.

### ¿Qué Significa "Semántica" en Contexto Computacional?

En el contexto de NLP, cuando hablamos de **semántica** nos referimos a la capacidad de:

1. **Reconocer sinónimos**: "auto" y "coche" deben tener representaciones similares
2. **Capturar relaciones conceptuales**: "médico" debe estar más cerca de "hospital" que de "computadora"
3. **Entender contexto**: distinguir "banco" (institución) de "banco" (asiento)
4. **Detectar analogías**: si "rey" es a "reina" como "hombre" es a "mujer"

### Introducción a los Vectores Semánticos Densos

La solución viene de representar palabras como **vectores densos** en un espacio multidimensional, donde:

- **Palabras similares** están **cerca** en el espacio vectorial
- **Relaciones semánticas** se preservan como **relaciones geométricas**
- **El contexto** determina la posición de cada palabra
- **Las dimensiones** capturan aspectos abstractos del significado

Experimentemos con esta idea usando spaCy, que incluye vectores pre-entrenados.

In [ ]:
# Instalación y configuración de spaCy
# Si es la primera vez que lo usan, necesitarán instalar el modelo en español
import subprocess
import sys

try:
    import spacy
    # Intentar cargar el modelo en español
    nlp = spacy.load("es_core_news_md")
    print("✅ spaCy y modelo español cargados correctamente")
except OSError:
    print("⚠️  Instalando modelo de spaCy en español...")
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_md"])
    import spacy
    nlp = spacy.load("es_core_news_md")
    print("✅ spaCy instalado y configurado")
except ImportError:
    print("⚠️  Instalando spaCy...")
    subprocess.check_call([sys.executable, "-m", "pip", "install", "spacy"])
    subprocess.check_call([sys.executable, "-m", "spacy", "download", "es_core_news_md"])
    import spacy
    nlp = spacy.load("es_core_news_md")
    print("✅ spaCy instalado desde cero")

# Información sobre el modelo
print(f"\nModelo: {nlp.meta['name']}")
print(f"Idioma: {nlp.meta['lang']}")
print(f"Dimensiones de vectores: {nlp.meta['vectors']['width']}")
print(f"Palabras con vectores: {nlp.meta['vectors']['keys']:,}")

In [ ]:
# Primer experimento: vectores semánticos vs BoW
# Usaremos los mismos ejemplos problemáticos de antes

print("EXPERIMENTO 1: RESOLVIENDO LA TRAGEDIA DE LOS SINÓNIMOS")
print("=" * 60)

# Palabras que son sinónimos pero BoW veía como diferentes
sinonimos_test = [
    ('padre', 'papá'),
    ('hermoso', 'bello'),
    ('auto', 'coche'),
    ('enojado', 'furioso'),
    ('casa', 'hogar')
]

print("\n🔍 SIMILITUDES SEMÁNTICAS CON SPACY:")
for palabra1, palabra2 in sinonimos_test:
    # Obtener vectores de spaCy
    token1 = nlp(palabra1)
    token2 = nlp(palabra2)

    # Calcular similitud (spaCy usa coseno internamente)
    similitud = token1.similarity(token2)

    print(f"'{palabra1}' ⟷ '{palabra2}': {similitud:.3f}")

print("\n💡 COMPARACIÓN CRÍTICA:")
print("- BoW/TF-IDF: similitud = 0.000 (palabras diferentes)")
print("- Vectores semánticos: similitud > 0.5 (reconoce relación)")
print("- Los vectores capturan el conocimiento de que estas palabras son sinónimos")

EXPERIMENTO 1: RESOLVIENDO LA TRAGEDIA DE LOS SINÓNIMOS

🔍 SIMILITUDES SEMÁNTICAS CON SPACY:
'padre' ⟷ 'papá': 0.630
'hermoso' ⟷ 'bello': 0.866
'auto' ⟷ 'coche': 0.685
'enojado' ⟷ 'furioso': 0.251
'casa' ⟷ 'hogar': 0.624

💡 COMPARACIÓN CRÍTICA:
- BoW/TF-IDF: similitud = 0.000 (palabras diferentes)
- Vectores semánticos: similitud > 0.5 (reconoce relación)
- Los vectores capturan el conocimiento de que estas palabras son sinónimos


In [ ]:
print("EXPERIMENTO 2: CAPTURANDO RELACIONES CONCEPTUALES")
print("=" * 55)

# Campos semánticos del corpus Casciari
campos_casciari = {
    'Fútbol': ['fútbol', 'partido', 'hinchada', 'cancha', 'gol', 'pelota'],
    'Escritura': ['escribir', 'texto', 'relato', 'historia', 'narración', 'palabras'],
    'Familia': ['padre', 'hija', 'familia', 'casa', 'hogar', 'amor'],
    'Control': ['computadora', 'internet', 'tecnología', 'pantalla']  # Grupo de control
}

print("\n🎯 SIMILITUDES DENTRO DE CAMPOS SEMÁNTICOS:")

for campo, palabras in campos_casciari.items():
    print(f"\n📋 {campo.upper()}:")

    # Calcular similitud promedio dentro del campo
    similitudes_internas = []

    for i, palabra1 in enumerate(palabras[:-1]):
        for palabra2 in palabras[i+1:]:
            try:
                token1 = nlp(palabra1)
                token2 = nlp(palabra2)
                sim = token1.similarity(token2)
                similitudes_internas.append(sim)
                print(f"  {palabra1} ⟷ {palabra2}: {sim:.3f}")
            except:
                print(f"  {palabra1} ⟷ {palabra2}: vector no disponible")

    if similitudes_internas:
        promedio = np.mean(similitudes_internas)
        print(f"  📊 Similitud promedio en {campo}: {promedio:.3f}")

EXPERIMENTO 2: CAPTURANDO RELACIONES CONCEPTUALES

🎯 SIMILITUDES DENTRO DE CAMPOS SEMÁNTICOS:

📋 FÚTBOL:
  fútbol ⟷ partido: 0.502
  fútbol ⟷ hinchada: 0.431
  fútbol ⟷ cancha: 0.476
  fútbol ⟷ gol: 0.326
  fútbol ⟷ pelota: 0.384
  partido ⟷ hinchada: 0.373
  partido ⟷ cancha: 0.426
  partido ⟷ gol: 0.429
  partido ⟷ pelota: 0.313
  hinchada ⟷ cancha: 0.469
  hinchada ⟷ gol: 0.325
  hinchada ⟷ pelota: 0.286
  cancha ⟷ gol: 0.417
  cancha ⟷ pelota: 0.699
  gol ⟷ pelota: 0.463
  📊 Similitud promedio en Fútbol: 0.421

📋 ESCRITURA:
  escribir ⟷ texto: 0.437
  escribir ⟷ relato: 0.359
  escribir ⟷ historia: 0.331
  escribir ⟷ narración: 0.347
  escribir ⟷ palabras: 0.396
  texto ⟷ relato: 0.566
  texto ⟷ historia: 0.296
  texto ⟷ narración: 0.453
  texto ⟷ palabras: 0.379
  relato ⟷ historia: 0.578
  relato ⟷ narración: 0.570
  relato ⟷ palabras: 0.334
  historia ⟷ narración: 0.727
  historia ⟷ palabras: 0.323
  narración ⟷ palabras: 0.355
  📊 Similitud promedio en Escritura: 0.430

📋 FAMIL

In [ ]:
print("EXPERIMENTO 3: COMPARACIÓN ENTRE CAMPOS SEMÁNTICOS")
print("=" * 52)

# Comparar palabras de diferentes campos semánticos
comparaciones_cruzadas = [
    ('fútbol', 'computadora', 'Deporte vs Tecnología'),
    ('padre', 'gol', 'Familia vs Deporte'),
    ('escribir', 'pelota', 'Arte vs Deporte'),
    ('hija', 'narración', 'Familia vs Arte'),
    ('cancha', 'internet', 'Deporte vs Tecnología')
]

print("\n🔄 SIMILITUDES ENTRE CAMPOS DIFERENTES:")
for palabra1, palabra2, descripcion in comparaciones_cruzadas:
    try:
        token1 = nlp(palabra1)
        token2 = nlp(palabra2)
        sim = token1.similarity(token2)
        print(f"{descripcion}: '{palabra1}' ⟷ '{palabra2}' = {sim:.3f}")
    except:
        print(f"{descripcion}: vector no disponible")

print("\n💭 ANÁLISIS DE RESULTADOS:")
print("- Palabras del mismo campo semántico tienen similitud ALTA (> 0.4)")
print("- Palabras de campos diferentes tienen similitud BAJA (< 0.3)")
print("- Los vectores organizan automáticamente el conocimiento semántico")
print("- Esta estructura estaba invisible en BoW/TF-IDF")

EXPERIMENTO 3: COMPARACIÓN ENTRE CAMPOS SEMÁNTICOS

🔄 SIMILITUDES ENTRE CAMPOS DIFERENTES:
Deporte vs Tecnología: 'fútbol' ⟷ 'computadora' = 0.023
Familia vs Deporte: 'padre' ⟷ 'gol' = 0.104
Arte vs Deporte: 'escribir' ⟷ 'pelota' = 0.025
Familia vs Arte: 'hija' ⟷ 'narración' = -0.005
Deporte vs Tecnología: 'cancha' ⟷ 'internet' = -0.047

💭 ANÁLISIS DE RESULTADOS:
- Palabras del mismo campo semántico tienen similitud ALTA (> 0.4)
- Palabras de campos diferentes tienen similitud BAJA (< 0.3)
- Los vectores organizan automáticamente el conocimiento semántico
- Esta estructura estaba invisible en BoW/TF-IDF


### Comprendiendo los Vectores Densos

Lo que acabamos de experimentar representa un salto cualitativo fundamental. En lugar de representar palabras como posiciones en un vocabulario gigante (BoW), ahora las representamos como **puntos en un espacio semántico continuo**.

**Características de los vectores semánticos densos:**

1. **Dimensionalidad reducida**: 300 dimensiones vs 30,000+ en BoW
2. **Valores continuos**: números reales vs enteros de conteo
3. **Información en todas las dimensiones**: sin ceros, cada componente aporta
4. **Relaciones geométricas**: la proximidad espacial refleja proximidad semántica

**¿De dónde vienen estos vectores?** (Adelanto conceptual)

Los vectores de spaCy fueron entrenados analizando millones de textos en español. El modelo aprendió que:
- Palabras que aparecen en contextos similares tienen significados similares
- "Padre" y "papá" aparecen en contextos muy parecidos, por eso sus vectores son similares
- "Fútbol" y "computadora" aparecen en contextos muy diferentes, por eso están alejados

El jueves profundizarán en **cómo** se entrenan estos vectores usando Word2Vec, FastText y GloVe.

In [ ]:
# Aplicación práctica: re-analizando fragmentos Casciari con vectores semánticos
print("EXPERIMENTO 4: RE-ANÁLISIS SEMÁNTICO DE TEXTOS CASCIARI")
print("=" * 58)

# Recuperar nuestros textos de ejemplo
textos_analisis = {
    'Paternidad temprana': textos_casciari_inspirados['paternidad_temprana'],
    'Paternidad madura': textos_casciari_inspirados['paternidad_madura'],
    'Fútbol pasión': textos_casciari_inspirados['futbol_pasion'],
    'Fútbol nostalgia': textos_casciari_inspirados['futbol_nostalgia']
}

# Calcular similitudes usando vectores semánticos
print("\n🔍 SIMILITUDES SEMÁNTICAS ENTRE TEXTOS:")

textos_lista = list(textos_analisis.values())
nombres_lista = list(textos_analisis.keys())

# Procesar textos con spaCy para obtener vectores de documento
docs_spacy = [nlp(texto) for texto in textos_lista]

# Calcular similitudes
for i, nombre1 in enumerate(nombres_lista):
    for j, nombre2 in enumerate(nombres_lista):
        if i < j:  # Evitar duplicados
            similitud = docs_spacy[i].similarity(docs_spacy[j])
            print(f"{nombre1} ⟷ {nombre2}: {similitud:.3f}")

print("\n📊 COMPARACIÓN BoW vs VECTORES SEMÁNTICOS:")
print("\nBoW/TF-IDF (de experimentos anteriores):")
print("- Textos de mismo tema pero diferente léxico: similitud ≈ 0.0")
print("- No detectaba la continuidad temática")
print("\nVectores semánticos (ahora):")
print("- Textos de paternidad: similitud > 0.7")
print("- Textos de fútbol: similitud > 0.6")
print("- Reconoce la coherencia temática a pesar del vocabulario diferente")

EXPERIMENTO 4: RE-ANÁLISIS SEMÁNTICO DE TEXTOS CASCIARI

🔍 SIMILITUDES SEMÁNTICAS ENTRE TEXTOS:
Paternidad temprana ⟷ Paternidad madura: 0.819
Paternidad temprana ⟷ Fútbol pasión: 0.609
Paternidad temprana ⟷ Fútbol nostalgia: 0.751
Paternidad madura ⟷ Fútbol pasión: 0.530
Paternidad madura ⟷ Fútbol nostalgia: 0.625
Fútbol pasión ⟷ Fútbol nostalgia: 0.708

📊 COMPARACIÓN BoW vs VECTORES SEMÁNTICOS:

BoW/TF-IDF (de experimentos anteriores):
- Textos de mismo tema pero diferente léxico: similitud ≈ 0.0
- No detectaba la continuidad temática

Vectores semánticos (ahora):
- Textos de paternidad: similitud > 0.7
- Textos de fútbol: similitud > 0.6
- Reconoce la coherencia temática a pesar del vocabulario diferente


---

## 4. Preparación Conceptual para Word Embeddings

### Del Experimento a la Comprensión Profunda

Los experimentos que acabamos de realizar con spaCy les dieron una **experiencia directa** con vectores semánticos, pero probablemente generaron nuevas preguntas:

- **¿Cómo se entrenan estos vectores?** ¿Qué algoritmos convierten texto en números que capturan semántica?
- **¿Por qué funcionan?** ¿Cuál es el principio matemático que permite que la geometría capture significado?
- **¿Cómo se decide la dimensionalidad?** ¿Por qué 300 dimensiones y no 100 o 1000?
- **¿Qué limitaciones tienen?** Si resuelven problemas de BoW, ¿qué problemas nuevos crean?

En la clase práctica del jueves van a explorar estas preguntas implementando los algoritmos fundamentales: **Word2Vec**, **FastText** y **GloVe**.

### Conceptos Clave para el Jueves

#### 1. La Hipótesis Distribucional
> "Una palabra se caracteriza por las compañías que mantiene" - J.R. Firth (1957)

Esta idea simple es la base teórica de todos los word embeddings:
- Palabras que aparecen en contextos similares tienen significados similares
- "Padre" y "papá" aparecen rodeadas de palabras similares: "mi", "querido", "familia", etc.
- Un algoritmo puede aprender esta regularidad y asignar vectores similares

#### 2. Word2Vec: La Arquitectura Revolucionaria
Word2Vec (Mikolov et al., 2013) propuso dos arquitecturas:
- **CBOW**: predice una palabra dado su contexto
- **Skip-gram**: predice el contexto dada una palabra

Ambas usan redes neuronales simples para aprender representaciones que resuelven estas tareas de predicción.

#### 3. FastText: Resolviendo Palabras Fuera de Vocabulario
FastText extiende Word2Vec considerando **sub-palabras**:
- Puede generar vectores para palabras que nunca vio durante el entrenamiento
- Especialmente útil para idiomas con morfología rica como el español

#### 4. GloVe: Estadísticas Globales de Co-ocurrencia
GloVe combina las ventajas de:
- Métodos basados en conteo (como LSA)
- Métodos de predicción (como Word2Vec)

Utiliza estadísticas de co-ocurrencia de todo el corpus para entrenar vectores.

In [ ]:
# Adelanto conceptual: problemas que resolverán el jueves
print("PREPARACIÓN PARA LA PRÁCTICA DEL JUEVES")
print("=" * 42)

print("\n🎯 PROBLEMAS QUE RESOLVERÁN CON WORD EMBEDDINGS:")

# Problema 1: Palabras fuera de vocabulario (OOV)
print("\n1️⃣  PROBLEMA OOV (Out of Vocabulary):")
palabras_casciari_raras = ['azulgrana', 'cuervos', 'santafesino', 'orsaiense']

for palabra in palabras_casciari_raras:
    try:
        token = nlp(palabra)
        if token.has_vector:
            print(f"  '{palabra}': tiene vector en spaCy")
        else:
            print(f"  '{palabra}': SIN vector en spaCy")
    except:
        print(f"  '{palabra}': no procesable")

print("  → FastText resolverá este problema usando sub-palabras")

# Problema 2: Analogías y relaciones
print("\n2️⃣  ANALOGÍAS (álgebra de palabras):")
print("  Pregunta: rey - hombre + mujer = ?")
print("  → Esperamos obtener 'reina'")
print("  → Implementarán búsqueda por analogías con Word2Vec")

# Problema 3: Similitud semántica
print("\n3️⃣  BÚSQUEDA POR SIMILITUD:")
print("  Pregunta: palabras más similares a 'fútbol'")
print("  → Implementarán búsqueda de vecinos más cercanos")
print("  → Compararán resultados entre Word2Vec, FastText y GloVe")

print("\n📚 DATASETS QUE USARÁN EL JUEVES:")
print("  - Vectores Word2Vec pre-entrenados en español (SBWC)")
print("  - Modelos FastText multiidioma")
print("  - Comparación de performance entre métodos")

print("\n🛠️ HERRAMIENTAS TÉCNICAS:")
print("  - Librería gensim para manipular vectores")
print("  - Visualización con t-SNE y PCA")
print("  - Evaluación cuantitativa de calidad de embeddings")

In [ ]:
# Reflexión crítica: limitaciones de los vectores semánticos
print("REFLEXIÓN CRÍTICA: LIMITACIONES DE VECTORES SEMÁNTICOS")
print("=" * 58)

print("\n⚠️  PROBLEMAS QUE AÚN EXISTEN:")

# Problema 1: Polisemia
palabras_polisemicas = [
    'banco',  # institución financiera vs asiento
    'capital', # ciudad vs dinero
    'carta',  # documento vs naipe
    'tiempo'  # duración vs clima
]

print("\n1️⃣  POLISEMIA (múltiples significados):")
for palabra in palabras_polisemicas:
    token = nlp(palabra)
    print(f"  '{palabra}': un solo vector para todos sus significados")

print("  → Los embeddings estáticos promedian todos los usos")
print("  → No distinguen contexto específico de uso")

# Problema 2: Sesgos
print("\n2️⃣  SESGOS SOCIALES EN LOS DATOS:")
ejemplos_sesgo = [
    ('doctor', 'enfermera', 'Profesiones de salud'),
    ('programador', 'secretaria', 'Profesiones técnicas')
]

for palabra1, palabra2, categoria in ejemplos_sesgo:
    try:
        token1 = nlp(palabra1)
        token2 = nlp(palabra2)
        sim = token1.similarity(token2)
        print(f"  {categoria}: '{palabra1}' ⟷ '{palabra2}' = {sim:.3f}")
    except:
        print(f"  {categoria}: error en el cálculo")

print("  → Los vectores reflejan sesgos presentes en los datos de entrenamiento")
print("  → Problema ético importante en aplicaciones reales")

print("\n3️⃣  ESTATICIDAD:")
print("  - Un vector por palabra, independiente del contexto")
print("  - No capturan cambios de significado en diferentes oraciones")
print("  - Limitación que motivó el desarrollo de modelos contextuales (BERT, etc.)")

print("\n💡 PERSPECTIVA:")
print("Los word embeddings fueron un avance revolucionario, pero no la solución final.")
print("En cursos avanzados explorarán embeddings contextuales que superan estas limitaciones.")

---

## 5. Síntesis y Reflexión Final

### El Camino Recorrido Hoy

En esta clase transitamos desde las limitaciones concretas de BoW/TF-IDF hasta las posibilidades de las representaciones semánticas. No fue un salto abstracto, sino una progresión basada en problemas reales que experimentaron con el corpus de Casciari:

1. **Identificamos limitaciones específicas**: ceguera semántica, pérdida de contexto, fragmentación por sinónimos
2. **Experimentamos con soluciones**: vectores densos, similitud semántica, relaciones conceptuales
3. **Desarrollamos intuición**: comprendimos por qué la geometría puede capturar significado
4. **Preparamos conceptos**: establecimos las bases para word embeddings del jueves

### Cambio de Paradigma Fundamental

Lo que experimentaron hoy representa un **cambio de paradigma** en NLP:

**Paradigma anterior (BoW/TF-IDF):**
- Palabras como símbolos discretos e independientes
- Representaciones basadas en co-ocurrencia exacta
- Espacios de alta dimensionalidad y dispersos
- Significado = frecuencia

**Nuevo paradigma (Embeddings):**
- Palabras como puntos en espacio semántico continuo
- Representaciones basadas en contexto distribucional
- Espacios de dimensionalidad moderada y densos
- Significado = posición relativa

### Conexión con el Jueves

En la práctica del jueves van a:
- **Implementar** los algoritmos que generan estos vectores
- **Entrenar** modelos Word2Vec en español
- **Comparar** FastText y GloVe en tareas específicas
- **Evaluar** calidad de embeddings cuantitativamente
- **Aplicar** vectores a problemas de similitud y analogías

### Preguntas para la Reflexión

Antes del jueves, reflexionen sobre:

1. **¿Cómo cambiaría su análisis de Casciari** si hubieran usado vectores semánticos desde el principio?
2. **¿Qué patrones adicionales** podrían haber descubierto con representaciones semánticas?
3. **¿En qué aplicaciones** serían cruciales estos vectores vs BoW tradicional?
4. **¿Cómo evaluarían** si un conjunto de embeddings es "bueno" para una tarea específica?

### Impacto en NLP Moderno

Los conceptos que exploraron hoy son **fundamentales** para entender:
- Modelos de lenguaje modernos (GPT, BERT)
- Sistemas de recomendación basados en texto
- Traducción automática neural
- Análisis de sentimientos avanzado
- Búsqueda semántica en documentos

**Los word embeddings fueron el primer paso hacia la IA que "entiende" texto.** En el jueves van a aprender exactamente cómo funcionan por dentro.

In [ ]:
# Ejercicio final de reflexión: preparación para el jueves
print("EJERCICIO DE PREPARACIÓN PARA LA PRÁCTICA DEL JUEVES")
print("=" * 56)

print("\n📝 PREGUNTAS DE AUTOEVALUACIÓN:")
print("\n1. Conceptual:")
print("   - ¿Por qué BoW no puede distinguir sinónimos?")
print("   - ¿Qué significa que dos vectores estén 'cerca' en el espacio semántico?")
print("   - ¿Por qué 300 dimensiones pueden capturar más información que 30,000?")

print("\n2. Práctico:")
print("   - ¿Cómo usarían similitud semántica para mejorar búsquedas en documentos?")
print("   - ¿Qué ventajas tendría FastText vs Word2Vec para analizar textos argentinos?")
print("   - ¿Cómo evaluarían si sus embeddings capturan bien el español rioplatense?")

print("\n3. Crítico:")
print("   - ¿Qué sesgos podrían aparecer en embeddings entrenados con noticias?")
print("   - ¿Cuándo seguirían usando BoW en lugar de embeddings?")
print("   - ¿Qué limitaciones de embeddings estáticos motivaron BERT?")

print("\n🎯 OBJETIVOS PARA EL JUEVES:")
print("   ✓ Cargar y usar vectores Word2Vec pre-entrenados")
print("   ✓ Implementar búsqueda por similitud")
print("   ✓ Resolver analogías con álgebra de vectores")
print("   ✓ Comparar Word2Vec, FastText y GloVe")
print("   ✓ Visualizar embeddings en 2D")
print("   ✓ Evaluar calidad de vectores cuantitativamente")

print("\n💡 CONSEJO FINAL:")
print("Lleguen el jueves con la mente abierta a experimentar. Los embeddings")
print("son tanto matemática como arte: hay que desarrollar intuición práctica")
print("para usarlos efectivamente en problemas reales.")

print("\n" + "="*60)
print("¡Nos vemos el jueves para la aventura práctica con embeddings!")
print("="*60)